In [1]:
import numpy as np
from scipy.stats import rayleigh
from scipy.special import digamma,erf
import scipy.io as sio
import pandas as pd

In [2]:
def parseMat(data):
    dataSeqs= data['Seqs']  
    mdtype = dataSeqs.dtype
    seqs = {n: dataSeqs[n].flatten() for n in mdtype.names}
    dataStats = data['Stats']  
    mdtype = dataSeqs.dtype  
    stats = {n: dataSeqs[n].flatten() for n in mdtype.names}
    return seqs,stats

In [3]:
class Model:
    def __init__(self,seqs,K):
            self.N = len(seqs['Time'])
            D= np.zeros(self.N)
            sigma = np.zeros(self.N)
            Tmax = np.zeros(self.N)
            self.K = K
            for i in range(self.N):
                D[i] = np.max(seqs['Mark'][i])
                sigma[i] = (4*(np.std(seqs['Time'][i])**5) / (3* len(seqs['Time'][i])))**0.2
                Tmax[i] = seqs['Time'][i][-1][0]
            self.D = np.int(np.max(D))
            Tmax = np.mean(Tmax)
            self.w = np.mean(sigma)
            self.landmark = self.w* range(np.int(np.ceil(Tmax/self.w)))
            self.alpha = np.ones(K)
            M = len(self.landmark)
            self.beta = np.ones((self.D,M,self.K,self.D)) / (M*(self.D**2))
            self.b = np.ones((self.D,K))/self.D
#             label = np.round(K * np.random.rand(self.N))

            l1 = np.zeros(int(self.N/2)+1)
            l2 = np.ones(int(self.N-self.N/2))
            label = np.append(l1, l2)

            self.r = np.zeros((self.N,K))
            for k in range(K):
                self.r[label==k,k] = 1
            self.kernel = "gauss"
class Alg:
    def __init__ (self, outer = 8,rho = 0.1,inner = 5,thres =  1e-5,Tmax = []):
        self.outer = outer
        self.rho = rho
        self.inner = inner
        self.thres = thres
        self.Tmax = Tmax

In [4]:
linkedInData = sio.loadmat('Data/LinkedinData.mat')
IPTVData = sio.loadmat('Data/IPTVData.mat')
IPTVSeqs,IPTVStats = parseMat(IPTVData)
linkedInSeqs,linedInStats = parseMat(linkedInData)

In [5]:
def kernel_int(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance = distance.astype(np.float64)
    landmark = landmark.astype(np.float64)
    distance -= landmark
    G = 0
    if model.kernel == 'gauss':
        G = 0.5 *(erf(distance/(np.sqrt(2))*model.w)
                  + erf(landmark/(np.sqrt(2))*model.w))
    elif model.kernel == 'exp':
        G = 1 - np.exp(-model.w * (distance-landmark));
        G[G<0] = 0
    return G

In [6]:
def kernel(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance = distance.astype(np.float64)
    landmark = landmark.astype(np.float64)
    distance -= landmark
    g = 0
    if model.kernel == 'gauss':
        g = np.exp(-(distance**2)/(2*(model.w**2))/(np.sqrt(2*np.pi)*model.w))
    elif model.kernel == 'exp':
        g = model.w * np.exp(-model.w * distance)
        g[g>1] = 0
    return g

In [7]:
def E_log_pi(alpha):
    return digamma(alpha)  - digamma(np.sum(alpha))

In [8]:
def Expectation_DMHP(seqs,model,alg):
    Nk = np.sum(model.r,0)
    alpha = model.alpha + Nk
    LL = E_log_pi(alpha)
    EX = np.zeros((model.N,model.K))
    for c in range(model.N):
        time = seqs['Time'][c].flatten()
        event = seqs['Mark'][c].flatten()
        Tstart = seqs['Start'][c].flatten()
        if not alg.Tmax:
            Tstop = seqs['Stop'][c].flatten()
        else:
            Tstop = alg.Tmax
            indt = time < alg.Tmax
            time = time[indt]
            event = event[indt]
        N = len(time)
        G = kernel_int(Tstop-time,model)
        for i in range(N):
            ui = event[i]
            ti = time[i]
        
            E_lambda_i = (np.sqrt(np.pi/2) * model.b[ui-1,:]).flatten()
            V_lambda_i = (2 - np.sqrt(np.pi/2) * (model.b[ui-1,:]**2)).flatten()
            if i > 0:
                uj = event[0:i].flatten()
                tj = time[0:i].flatten()
                gij = kernel(ti-tj,model)
                auiuj = model.beta[uj-1,:,:,uj-1]
                tiled = np.tile(gij,(1,1,1))
                tiled = np.tile(tiled.T,(1,1,model.K))
                pij = np.multiply(tiled,auiuj)
            
                tmp = np.sum(pij,axis=(0,1))
                E_lambda_i += tmp
                tmp = np.sum(pij**2,axis=(0,1))
                V_lambda_i += tmp
            LL += np.log(E_lambda_i) - (V_lambda_i/ (2*(E_lambda_i**2)))
        LL -= (Tstop-Tstart).flatten() * (np.sqrt(np.pi/2) * np.sum(model.b))
        temp = np.tile(G,(model.K,1,1)).T
        temp2 = np.multiply(temp,np.sum(model.beta[event-1,:,:,:],axis = 3))
        tmp = np.sum(temp2,(0,1))
        LL -= tmp
        XX = (LL - np.max(LL))
        EX[c,:] = (np.exp(XX) / np.sum(np.exp(XX)))
    model.r = EX

In [1]:
def Maximization_DMHP (seqs,model,alg):
    EX = model.r
    A = model.beta
    mu = np.sqrt(np.pi/2)*model.b

    for inn in range(alg.inner):
        temp1 = A / model.beta
        np.nan_to_num(temp1,False)
        temp1[np.isinf(temp1)]  = 0
    
        temp2 = (mu**2) / (2*(model.b**2))
        np.nan_to_num(temp2,False)
        temp2[np.isinf(temp2)]  = 0
    
        temp3 = np.log(mu)  
        np.nan_to_num(temp3,False)
        temp2[np.isinf(temp3)]  = 0
    
        NLL = np.sum(temp1) + np.sum(temp2) + np.sum(temp3)
    
        MuA = 1 / (model.b**2)
        MuA[np.isinf(MuA)]  = 0
        MuB = 0
        MuC = - np.ones(model.b.shape)
    
        AB = np.zeros(A.shape)
        AA =  1 / (model.beta)
        AA[np.isinf(AA)] = 0
    
        for c in range(model.N):
        
            time = seqs['Time'][c].flatten()
            event = seqs['Mark'][c].flatten()
            Tstart = seqs['Start'][c].flatten()
        
            if not alg.Tmax:
                Tstop = seqs['Stop'][c].flatten()
            else:
                Tstop = alg.Tmax
                indt = time < alg.Tmax
                time = time[indt]
                event = event[indt]
            N = len(time)
            G = kernel_int(Tstop-time,model)
        
            TMPAA = np.zeros(A.shape)
            TMPAB = np.zeros(A.shape)
            TMPMuC = np.zeros(mu.shape)
            LL = 0
        
            for i in range(N):
           
                ui = event[i]
                ti = time[i]
            
                tmp = np.tile(G[:,i],(model.D,model.K,1)).T
            
                TMPAA[ui-1,:,:,:] = TMPAA[ui-1,:,:,:] + tmp 
            
                lambdai = mu[ui-1,:]
                pii = lambdai;
            
                if i > 0:
                    uj = event[0:i].flatten()
                    tj = time[0:i].flatten()
                    gij = kernel(ti-tj,model)
                    auiuj = model.beta[uj-1,:,:,uj-1]
                    tiled = np.tile(gij,(1,1,1))
                    tiled = np.tile(tiled.T,(1,1,model.K))
                    pij = np.multiply(tiled,auiuj)
                
                    tmp = np.sum(pij,(0,1))
                    lambdai += tmp
                
                    tmp = np.tile(lambdai,(pij.shape[0],pij.shape[1],1))
                    pij /= tmp
                
                    for j in range(i):
                        uj = event[j]
                        TMPAB[uj-1,:,:,ui-1] -= pij[j,:,:]
                    
                LL += np.log(lambdai) 
            
                pii /= lambdai
                TMPMuC[ui-1,:] -= pii
            
            LL-= (Tstop - Tstart) * (np.sum(mu))
            temp = np.tile(G,(model.K,1,1)).T
            temp2 = np.multiply(temp,np.sum(A[event-1,:,:,:],axis = 3))
            tmp = np.sum(temp2,(0,1))
            LL -= tmp
        
            MuB += (Tstop-Tstart) * EX[c,:]
        
            for k in range(model.K):
                AA[:,:,k,:] = AA[:,:,k,:] + (EX[c,k]*TMPAA[:,:,k,:]) 
                AB[:,:,k,:] = AB[:,:,k,:] + (EX[c,k]*TMPAB[:,:,k,:]) 
                MuC[:,k] = MuC[:,k] +  (EX[c,k]*TMPMuC[:,k]) 
            NLL -= (EX[c,:]*LL)
        MuBB = np.tile(MuB,(model.D,1))
        mutmp = (-MuBB + np.sqrt((MuBB)**2 - 4*(MuA * MuC))) / (2*MuA)
        Atmp = -AB / AA
    
        np.nan_to_num(Atmp,True)
        np.nan_to_num(mutmp,True)
        Atmp[np.isinf(Atmp)] = 0
        mutmp[np.isinf(mutmp)] = 0
    
        Err = np.sum(np.abs(A - Atmp)) / np.sum(np.abs(A))
        print("Inner = ",inn, " NLL = ", NLL, " Err = ",Err)
    
        A = Atmp
        mu = mutmp
    
        if Err < alg.thres or  inn == alg.inner:
            break
    model.beta = A
    model.b = np.sqrt(2/np.pi) * mu
        

In [9]:
for key in linkedInSeqs:
    print(key, len(linkedInSeqs[key]))

Time 2439
Mark 2439
Start 2439
Stop 2439
Feature 2439


In [10]:
model = Model(linkedInSeqs, 2)
alg = Alg(8, 0.1, 5, 1e-5, [])

In [11]:
model.r

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [12]:
Expectation_DMHP(linkedInSeqs,model,alg)

In [13]:
print(model.r)

[[ 0.50020492  0.49979508]
 [ 0.50020492  0.49979508]
 [ 0.50020492  0.49979508]
 ..., 
 [ 0.50020492  0.49979508]
 [ 0.50020492  0.49979508]
 [ 0.50020492  0.49979508]]


In [14]:
Expectation_DMHP(linkedInSeqs,model,alg)

In [15]:
print(model.r)

[[ 0.50020483  0.49979517]
 [ 0.50020483  0.49979517]
 [ 0.50020483  0.49979517]
 ..., 
 [ 0.50020483  0.49979517]
 [ 0.50020483  0.49979517]
 [ 0.50020483  0.49979517]]
